In [0]:
!pip install flair

     |████████████████████████████████| 143kB 2.6MB/s 
     |████████████████████████████████| 798kB 58.1MB/s 
     |████████████████████████████████| 1.0MB 59.2MB/s 
     |████████████████████████████████| 235kB 74.5MB/s 
     |████████████████████████████████| 501kB 70.2MB/s 
     |████████████████████████████████| 1.0MB 60.9MB/s 
     |████████████████████████████████| 870kB 63.0MB/s 
     |████████████████████████████████| 3.7MB 56.5MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=434982f9db0d0b9c1ea61fc67e9c8118a5e33d6bbba313fe7e736ec8f6ba759e
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993459 sha256=233a821744da57e11d42910e68f07781bb3a67dd0985c8753e5ce3d8f84cf643
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
  Created wheel for segtok: fil

In [0]:
from flair.datasets import ColumnCorpus

In [0]:
from flair.data import Corpus
from flair.embeddings import TokenEmbeddings, StackedEmbeddings, CharacterEmbeddings, FlairEmbeddings, BertEmbeddings
from typing import List
from flair.data import Sentence

In [0]:
corpus =ColumnCorpus(data_folder='/content/',
                           column_format={0: 'text', 1: 'ner'},
                           train_file='ner_data_train.conll',
                           test_file='ner_data_test.conll',
                           dev_file='ner_data_dev.conll')

2020-02-25 11:52:50,514 Reading data from /content
2020-02-25 11:52:50,516 Train: /content/ner_data_train.conll
2020-02-25 11:52:50,516 Dev: /content/ner_data_dev.conll
2020-02-25 11:52:50,517 Test: /content/ner_data_test.conll


In [0]:
print(corpus.train[0])

Sentence: "942840618 FIH 7113925 2524 641867 6/13/1994 12:00:00 AM CARDIOGENIC SHOCK . Unsigned DIS Report Status : Unsigned DISCHARGE SUMMARY NAME : RATA GIVEN , UNIT NUMBER : 126-10-12 ADMISSION DATE : 06/13/94 DISCHARGE DATE : 07/01/94 PRINCIPAL DIAGNOSIS : Cardiogenic shock . OTHER DIAGNOSES : 1 ) Coronary artery disease" - 50 Tokens


In [0]:
print(corpus)

Corpus: 4886 train + 630 dev + 828 test sentences


In [0]:
for entity in corpus.train[0].get_spans('ner'):
    print(entity)

TimeAttributes-span [6]: "6/13/1994"
TimeAttributes-span [7,8]: "12:00:00 AM"
Diseases-span [9,10]: "CARDIOGENIC SHOCK"
SectionHeader-span [14,15]: "Report Status"
SectionHeader-span [18,19,20]: "DISCHARGE SUMMARY NAME"
SectionHeader-span [29,30]: "ADMISSION DATE"
TimeAttributes-span [32]: "06/13/94"
SectionHeader-span [33,34]: "DISCHARGE DATE"
TimeAttributes-span [36]: "07/01/94"
SectionHeader-span [37,38]: "PRINCIPAL DIAGNOSIS"
Diseases-span [40,41]: "Cardiogenic shock"
SectionHeader-span [43,44]: "OTHER DIAGNOSES"
Diseases-span [48,49,50]: "Coronary artery disease"


In [0]:
tag_type = 'ner'

tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

Dictionary with 22 tags: <unk>, O, B-TimeAttributes, I-TimeAttributes, B-Diseases, I-Diseases, B-SectionHeader, I-SectionHeader, B-Procedure, I-Procedure, B-StatusCode, B-LabTests, B-Medicines, I-Medicines, B-Observations, I-Observations, I-LabTests, B-BodyParts, I-BodyParts, I-StatusCode, <START>, <STOP>


In [0]:
embedding_types: List[TokenEmbeddings] = [

    # init multilingual BERT
    BertEmbeddings('distilbert-base-uncased'),

    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)


2020-02-25 11:53:24,738 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-forward--h2048-l1-d0.05-lr30-0.25-20/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmprw8_mmue


100%|██████████| 73034624/73034624 [00:08<00:00, 8547705.57B/s]

2020-02-25 11:53:34,494 copying /tmp/tmprw8_mmue to cache at /root/.flair/embeddings/news-forward-0.4.1.pt
2020-02-25 11:53:34,561 removing temp file /tmp/tmprw8_mmue


2020-02-25 11:53:46,794 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-backward--h2048-l1-d0.05-lr30-0.25-20/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmp75wp8ie0


100%|██████████| 73034575/73034575 [00:08<00:00, 8473478.68B/s]

2020-02-25 11:53:56,627 copying /tmp/tmp75wp8ie0 to cache at /root/.flair/embeddings/news-backward-0.4.1.pt
2020-02-25 11:53:56,691 removing temp file /tmp/tmp75wp8ie0


In [0]:
# initialize sequence tagger
from flair.models import SequenceTagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# start training
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=20,
              embeddings_storage_mode='gpu')

2020-02-25 11:53:57,991 ----------------------------------------------------------------------------------------------------
2020-02-25 11:53:57,993 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): BertEmbeddings(
      (model): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0): TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(in_features=768, out_features=768, bias=True)
                (k_lin): Linear(in_features=768, out_features=768, bias=True)
                (v_lin): Linear(in_features=768, out_features=7

{'dev_loss_history': [tensor(25.9390, device='cuda:0'),
  tensor(22.0602, device='cuda:0'),
  tensor(17.2432, device='cuda:0'),
  tensor(14.7321, device='cuda:0'),
  tensor(14.9604, device='cuda:0'),
  tensor(12.6302, device='cuda:0'),
  tensor(11.7597, device='cuda:0'),
  tensor(11.6001, device='cuda:0'),
  tensor(11.1545, device='cuda:0'),
  tensor(12.0273, device='cuda:0'),
  tensor(12.3963, device='cuda:0'),
  tensor(10.9565, device='cuda:0'),
  tensor(10.7689, device='cuda:0'),
  tensor(10.6710, device='cuda:0'),
  tensor(9.9437, device='cuda:0'),
  tensor(10.5763, device='cuda:0'),
  tensor(10.0998, device='cuda:0'),
  tensor(9.5653, device='cuda:0'),
  tensor(9.4303, device='cuda:0'),
  tensor(9.7093, device='cuda:0')],
 'dev_score_history': [0.488,
  0.5705,
  0.6495,
  0.6996,
  0.694,
  0.7377,
  0.7352,
  0.7472,
  0.7503,
  0.7376,
  0.7334,
  0.7472,
  0.7658,
  0.7606,
  0.7651,
  0.7587,
  0.7656,
  0.7742,
  0.777,
  0.771],
 'test_score': 0.7753,
 'train_loss_history':

In [0]:
# load the model you trained
model = SequenceTagger.load('resources/taggers/example-ner/final-model.pt')

2020-02-25 12:06:01,602 loading file resources/taggers/example-ner/final-model.pt
